## Context
An international e-commerce company based wants to discover key insights from their customer database. They want to use some of the most advanced machine learning techniques to study their customers. The company sells electronic products.🚢

## Content
The dataset used for model building contained 10999 observations of 12 variables.
The data contains the following information:

* ID: ID Number of Customers.
* Warehouse block: The Company have big Warehouse which is divided in to block such as A,B,C,D,E.
* Mode of shipment:The Company Ships the products in multiple way such as Ship, Flight and Road.
* Customer care calls: The number of calls made from enquiry for enquiry of the shipment.
* Customer rating: The company has rated from every customer. 1 is the lowest (Worst), 5 is the highest (Best).
* Cost of the product: Cost of the Product in US Dollars.
* Prior purchases: The Number of Prior Purchase.
* Product importance: The company has categorized the product in the various parameter such as low, medium, high.
* Gender: Male and Female.
* Discount offered: Discount offered on that specific product.
* Weight in gms: It is the weight in grams.
* Reached on time: It is the target variable, where 1 Indicates that the product has NOT reached on time and 0 indicates it has reached on time.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.core.display import HTML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale  # for normalizing data
import xgboost
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier as RT
from sklearn.naive_bayes import GaussianNB as GB  # for Naive Bayesian 
from sklearn.svm import SVC # Support Vendor Machine

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# load the file
ecomm = pd.read_csv('../input/customer-analytics/Train.csv')

# 📌 Explanatory Analysis & Visualization

In [3]:
ecomm.info()

In [4]:
ecomm.isnull().sum()

In [5]:
ecomm.sample(5)

In [6]:
ecomm[['Customer_care_calls','Customer_rating', 'Cost_of_the_Product','Prior_purchases','Discount_offered', 'Weight_in_gms']].describe()

In [7]:
ecomm.describe(include='object')

In [8]:
ecomm.sample(5)

### Frequency Distribution for Numeric Data 🙋‍♀️

In [9]:
# plotting multiple graphs in a grid
# exploring the distribution of numeric columns using cumulative frequency distribution
fig, axes = plt.subplots(3,1,figsize=(20,16))

# cumulative frequency distribution for'Cost_of_the_Product'
sns.histplot(ecomm.Cost_of_the_Product, kde=True, ax=axes[0])
ax=axes[0].set_title('Frequency Distribution of Product Cost',fontsize=12)
ax=axes[0].set_xlabel('Product Cost',fontsize=10)
ax=axes[0].set_ylabel('Count',fontsize=10)

# cumulative frequency distribution for 'Weight_in_gms'
sns.histplot(ecomm.Weight_in_gms, kde=True, ax=axes[1])
ax=axes[1].set_title('Frequency Distribution of Weight',fontsize=12)
ax=axes[1].set_xlabel('Weight (in gms)',fontsize=10)
ax=axes[1].set_ylabel('Count',fontsize=10)

# cumulative frequency distribution for'Discount_offered'
sns.histplot(ecomm.Discount_offered, kde=True, ax=axes[2])
ax=axes[2].set_title('Frequency Distribution of Discount Offered',fontsize=12)
ax=axes[2].set_xlabel('Discount Offered',fontsize=10)
ax=axes[2].set_ylabel('Count',fontsize=10)

#### From the cumulative frequency distribution above, we can infer that:
<ol>
    <li>The cost of most of the products falls within the range of 240.00 - 275.00 dollars.</li>
    <li>The weight of most of the products falls within the range of 1,000-2,000gms and 4,000-6,000gms. </li>
    <li>The discount that is most likely to be given is between 1% to 10%. There are many outliers located at the third quartile. </li>
</ol>

### Countplots for Categorical Data 💁‍♀️

In [10]:
# for categorical data
fig, axes = plt.subplots(4,2,figsize=(16,25),facecolor='#e6ffff')

# countplot for 'Warehouse_block'
abs_whs=ecomm["Warehouse_block"].value_counts(ascending=False)
sns.countplot(x=ecomm["Warehouse_block"], order=abs_whs.index, ax=axes[0,0], palette='CMRmap_r')
axes[0,0].set_title('Orders Handled By Each Warehouse Block', fontsize=12)
axes[0,0].set_xlabel('Warehouse Block', fontsize=10)
axes[0,0].set_ylabel('Count', fontsize=10)
rel_whs=ecomm["Warehouse_block"].value_counts(ascending=False, normalize=True).values*100
lbs_whs=[f"{w[0]} ({w[1]:.2f}%)" for w in zip(abs_whs,rel_whs)]
axes[0,0].bar_label(container=axes[0,0].containers[0], labels=lbs_whs)

# countplot for 'Mode_of_Shipment' 
abs_ship = ecomm["Mode_of_Shipment"].value_counts(ascending=False) 
sns.countplot(x=ecomm["Mode_of_Shipment"], order=abs_ship.index, ax=axes[0,1], palette=['#DC143C','#556b2f','#008b8b'])
axes[0,1].set_title('Number of Orders By Shipment Mode', fontsize=12)
axes[0,1].set_xlabel('Shipment Mode', fontsize=10)
axes[0,1].set_ylabel('Count', fontsize=10)
rel_ship = ecomm["Mode_of_Shipment"].value_counts(ascending=False, normalize=True).values*100
lbs_ship = [f"{s[0]} ({s[1]:.2f}%)" for s in zip (abs_ship,rel_ship)]
axes[0,1].bar_label(container=axes[0,1].containers[0], labels=lbs_ship)

# countplot for 'Customer_care_calls'
abs_calls=ecomm["Customer_care_calls"].value_counts(ascending=False)
sns.countplot(x=ecomm["Customer_care_calls"], order=abs_calls.index,ax=axes[1,0],palette='cubehelix')
axes[1,0].set_title('Number of Customer Care Calls Made by Customers', fontsize=12)
axes[1,0].set_xlabel('Customer Care Calls', fontsize=10)
axes[1,0].set_ylabel('Count', fontsize=10)
rel_calls=ecomm["Customer_care_calls"].value_counts(ascending=False, normalize=True).values*100
lbs_calls=[f"{c[0]} ({c[1]:.2f}%)" for c in zip(abs_calls, rel_calls)]
axes[1,0].bar_label(container=axes[1,0].containers[0], labels=lbs_calls)

# countplot for 'Customer_rating'
abs_rating = ecomm["Customer_rating"].value_counts(ascending=False)
sns.countplot(x=ecomm["Customer_rating"], order=abs_rating.index,ax=axes[1,1],palette="rocket")
axes[1,1].set_title('Customer Rating Received', fontsize=12);
axes[1,1].set_xlabel('Customer Rating', fontsize=10)
axes[1,1].set_ylabel('Count', fontsize=10)
rel_rating = ecomm["Customer_rating"].value_counts(ascending=False, normalize=True).values*100
lbs_rating = [f"{r[0]} ({r[1]:.2f}%)" for r in zip(abs_rating, rel_rating)]
axes[1,1].bar_label(container=axes[1,1].containers[0], labels=lbs_rating)

# countplot for 'Prior_purchases'
abs_prior_pur = ecomm["Prior_purchases"].value_counts(ascending=False)
sns.countplot(x=ecomm["Prior_purchases"], order=abs_prior_pur.index,ax=axes[2,0],palette='viridis')
axes[2,0].set_title('Number of Prior Purchases Made by Customers', fontsize=12)
axes[2,0].set_xlabel('Prior Purchases', fontsize=10)
axes[2,0].set_ylabel('Count', fontsize=10)
rel_prior_pur = ecomm["Prior_purchases"].value_counts(ascending=False, normalize=True).values*100
lbs_prior_pur = [f"{pur[0]} ({pur[1]:.0f}%)" for pur in zip(abs_prior_pur, rel_prior_pur)]
axes[2,0].bar_label(container=axes[2,0].containers[0], labels=lbs_prior_pur)

# countplot for'Product_importance'
abs_priority = ecomm["Product_importance"].value_counts(ascending=False)
sns.countplot(x=ecomm["Product_importance"], order=abs_priority.index,ax=axes[2,1])
axes[2,1].set_title('Number of Orders Made by Product Importance', fontsize=12)
axes[2,1].set_xlabel('Product Importance', fontsize=10)
axes[2,1].set_ylabel('Count', fontsize=10)
rel_priority = ecomm["Product_importance"].value_counts(ascending=False, normalize=True).values*100
lbs_priority = [f"{i[0]} ({i[1]:.2f}%)" for i in zip(abs_priority, rel_priority)]
axes[2,1].bar_label(container=axes[2,1].containers[0], labels=lbs_priority)

# countplot for'Gender'
abs_gender = ecomm["Gender"].value_counts(ascending=False)
sns.countplot(x=ecomm["Gender"], order=abs_gender.index,ax=axes[3,0],palette=['#800000','#191970'])
axes[3,0].set_title("Number of Orders Made by Customers' Gender", fontsize=12)
axes[3,0].set_xlabel("Customers' Gender", fontsize=10)
axes[3,0].set_ylabel('Count', fontsize=10)
rel_gender = ecomm["Gender"].value_counts(ascending=False, normalize=True).values*100
lbs_gender = [f"{g[0]} ({g[1]:.2f}%)" for g in zip(abs_gender, rel_gender)]
axes[3,0].bar_label(container=axes[3,0].containers[0], labels=lbs_gender)

# countplot for'Reached.on.Time_Y.N'
abs_arrival = ecomm["Reached.on.Time_Y.N"].value_counts(ascending=False)
sns.countplot(x=ecomm["Reached.on.Time_Y.N"], order=abs_arrival.index,ax=axes[3,1],palette='tab20c_r')
axes[3,1].set_title('Number of Orders Based On Arrival Time', fontsize=12)
axes[3,1].set_xlabel('Arrival', fontsize=10)
axes[3,1].set_xticklabels(['Late', 'On Time'])
axes[3,1].set_ylabel('Count', fontsize=10)
rel_arrival = ecomm["Reached.on.Time_Y.N"].value_counts(ascending=False, normalize=True).values*100
lbls_arrival=[f"{a[0]}({a[1]:.2f}%)" for a in zip(abs_arrival,rel_arrival)]
axes[3,1].bar_label(container=axes[3,1].containers[0],labels=lbls_arrival);

#### From the countplots above, we can infer that:
1. Most of the orders were handled by warehouse block F, while other warehouse blocks handled the rest of the orders equally.
2. Most of the orders were shipped by ship, followed by flight, and finally by road.
3. Many of the customers needed to make 4 calls to track their shipment(s). This is a warning point for the company as the concern need to be solved as soon as possible without the need for the customers to make several calls. 
4. 1 is the rating with the second highest count. It is more likely due to the fact the products are not delivered on time and a high number of calls that the customers need to make. 
5. Many of the customers had made 3 prior purchases. And, we can infer that we have 128 loyal customers as they had made at least 8 (max=10) purchases previously.
6. Most of the orders are of low importance (48.16%), followed by medium (43.22%) and then high (8.62%).
7. Most of the customers are females (50.41%), while males made up 49.59% of the customers.
8. Out of the 10,999 orders, 6,563 orders were not delivered on time and it made up more than half of the orders (59.67%). 

# 📌Asking and Answering Questions 🧐

*Below are just the questions that I came out by myself and I think these might help the e-commerce company to determine the factors that caused the late orders.*

### Q1. What is the percentage of customers that are going to  ceases his or her relationship with the company? 

*Take into account of customers who made at least 4 customer care calls and gave rating of 1, typically focused more on customers that were given a discount of more than 14%, and he or she made at least 4 prior purchases.*

In [11]:
j = ecomm[(ecomm['Customer_rating']<=2) & (ecomm['Prior_purchases']>=4) & (ecomm['Customer_care_calls']>=4) & (ecomm['Discount_offered']>=14)]
print(f"{round(len(j) / len(ecomm.ID) * 100, 2)}% of the customers are likely to cease his or her relationship with the e-commerce company as the customers had made at least 4 customer care calls and gave rating of 1 although they made at least 4 prior purchases and was given discount of at least 14%.")

### Q2. What is the average cost of the product and the average discount is given to the customers whose orders are of high importance (categorized by customer rating) but arrived late?

In [12]:
five = ecomm[(ecomm['Product_importance']=='high') & (ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Customer_rating']==5)][['Prior_purchases','ID','Discount_offered','Cost_of_the_Product','Customer_rating']].groupby('Customer_rating').agg({'Prior_purchases':'mean','ID':'count','Discount_offered':'mean', 'Cost_of_the_Product':'mean'})
four = ecomm[(ecomm['Product_importance']=='high') & (ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Customer_rating']==4)][['Prior_purchases','ID','Discount_offered','Cost_of_the_Product','Customer_rating']].groupby('Customer_rating').agg({'Prior_purchases':'mean','ID':'count','Discount_offered':'mean', 'Cost_of_the_Product':'mean'})
three = ecomm[(ecomm['Product_importance']=='high') & (ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Customer_rating']==3)][['Prior_purchases','ID','Discount_offered','Cost_of_the_Product','Customer_rating']].groupby('Customer_rating').agg({'Prior_purchases':'mean','ID':'count','Discount_offered':'mean', 'Cost_of_the_Product':'mean'})
two = ecomm[(ecomm['Product_importance']=='high') & (ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Customer_rating']==2)][['Prior_purchases','ID','Discount_offered','Cost_of_the_Product','Customer_rating']].groupby('Customer_rating').agg({'Prior_purchases':'mean','ID':'count','Discount_offered':'mean', 'Cost_of_the_Product':'mean'})
one = ecomm[(ecomm['Product_importance']=='high') & (ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Customer_rating']==1)][['Prior_purchases','ID','Discount_offered','Cost_of_the_Product','Customer_rating']].groupby('Customer_rating').agg({'Prior_purchases':'mean','ID':'count','Discount_offered':'mean', 'Cost_of_the_Product':'mean'})
rating_discount_cost = pd.concat([five, four, three, two, one],axis=0).rename(columns={'ID':'Number of Highly Important But Late Orders'})
rating_discount_cost.sort_values('Number of Highly Important But Late Orders', ascending=False)

#### Based on the table above, we can infer that:
1. Most of the customers who had purchased previously would give a rating of 5 even their orders were late. This inference was made by looking at the differences in the number of highly important but late orders (4) and the difference in prior purchases (0.36) for rating 5 and rating 1. The difference is so small, and it suggests that there is other factors that led the customers to give rating 1, and it might be due to defective items received, bad packaging, or other factors.
2. The number of highly important but late orders for rating 1 is lower than that for rating 5, but the average product cost for rating 1 is also low. For high cost products, it will result in a higher average product cost after dividing the high total product cost with low number of highly important but late orders. However, in this case for rating 1, a different condition happened, whereby the customers were ordering low cost products and yet giving rating 1. Again, it suggests that there is other factors that led the customers to give rating 1, and more variables need to be added in the study.

### Q3. What is the percentage of highly important orders that arrived late? Which warehouse block resulted the highest number of highly important but late orders (in proportion of the total number of orders handled)?

In [13]:
fig, axes = plt.subplots(3,2,figsize=(16,16),facecolor='#ffffe6')

late_vs_importance = ecomm[ecomm['Reached.on.Time_Y.N']==1].groupby('Product_importance').agg({'Product_importance':'count'})
late_vs_importance.rename(columns={'Product_importance':'Number of Late Orders'},inplace=True)
a = sns.barplot(x=late_vs_importance.index, y='Number of Late Orders', data=late_vs_importance,ax=axes[0,0])
a.set(xlabel='Product Importance', ylabel = 'Number of Late Orders', title='Number of Late Orders by Product Importance')
a.bar_label(a.containers[0])

t =ecomm[ecomm['Reached.on.Time_Y.N']==1].groupby('Warehouse_block').agg({'Reached.on.Time_Y.N':'count'}).sort_values('Reached.on.Time_Y.N',ascending=False) 
t.rename(columns={'Reached.on.Time_Y.N':'Number of Late Orders'},inplace=True)
late_whs = t.reset_index()
b = sns.barplot(x='Warehouse_block', y='Number of Late Orders', data=late_whs, palette="icefire",ax=axes[0,1])
# b.set_xlabel('Warehouse Block', fontsize=12)
# b.set_ylabel('Number of Late Orders', fontsize=12)
# b.set_title('Number of Late Orders by Warehouse Block', fontsize=14)
b.set(xlabel='Warehouse Block', ylabel = 'Number of Late Orders', title='Number of Late Orders by Warehouse Block')
b.bar_label(b.containers[0])

# autopct: display the percentage of each segment we can define the format.
late_order_by_ship_mode = ecomm[ecomm['Reached.on.Time_Y.N']==1].groupby('Mode_of_Shipment').agg({'Mode_of_Shipment':'count'})
late_order_by_ship_mode.plot.pie(y='Mode_of_Shipment', 
                                 autopct='%1.2f%%', 
                                 colors=['darkolivegreen','darkcyan','crimson'],
                                 ylabel="",
                                 title='Number of Late Orders by Ship Mode',
                                 fontsize=12,
                                 ax=axes[1,0]);

s = ecomm[ecomm['Reached.on.Time_Y.N']==1][['Warehouse_block','Product_importance']].groupby(['Warehouse_block','Product_importance']).agg({'Product_importance':'count'})
s.rename(columns={'Product_importance':'Number of Late Orders'}, inplace=True)
whs_late_important = s.reset_index()
c = sns.barplot(x='Warehouse_block', y='Number of Late Orders', hue='Product_importance', data=whs_late_important,ax=axes[1,1])
c.set(xlabel='Warehouse Block', title='Product Importance of Late Orders by Warehouse Block')
c.bar_label(c.containers[0])
c.bar_label(c.containers[1])
c.bar_label(c.containers[2])

chosen_colors=['tomato','mediumaquamarine','orchid','darkgoldenrod', 'steelblue']
whs_late_orders = ecomm[(ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Product_importance']=='high')].groupby('Warehouse_block').agg({'Reached.on.Time_Y.N':'count'})
whs_late_orders.rename(columns={'Reached.on.Time_Y.N':'Number of Late Orders'},inplace=True)
whs_late_orders.plot.pie(y='Number of Late Orders', autopct='%1.2f%%', colors=chosen_colors, fontsize=10, ylabel="", ax=axes[2,0])
axes[2,0].set_title('Highly Important But Late Orders by Warehouse Block')

k=ecomm[(ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Product_importance']=='high')][['Warehouse_block','Mode_of_Shipment']].groupby(['Warehouse_block','Mode_of_Shipment']).agg({'Mode_of_Shipment':'count'})
k.rename(columns={'Mode_of_Shipment':'Number of Highly Important but Late Orders'},inplace=True)
whs_late_important_ship = k.reset_index()
v = sns.barplot(x='Warehouse_block', y='Number of Highly Important but Late Orders', hue='Mode_of_Shipment', data=whs_late_important_ship,ax=axes[2,1],palette=['#556b2f','#008b8b','#DC143C'])
v.set(xlabel='Warehouse Block', title='Highly Important but Late Orders by Warehouse Block & Ship Mode')
v.bar_label(v.containers[0])
v.bar_label(v.containers[1])
v.bar_label(v.containers[2]);

In [14]:
# total orders handled by each warehouse block
w = ecomm.groupby('Warehouse_block').agg({'ID':'count'}).sort_values('ID',ascending=False)
w.rename(columns={'ID':'Total Number of Orders Handled'}, inplace=True)

# total highly important orders handled by each warehouse block
x = ecomm[ecomm['Product_importance']=='high'].groupby('Warehouse_block').agg({'Reached.on.Time_Y.N':'count'})
x.rename(columns={'Reached.on.Time_Y.N':'Number of Highly Important Orders Handled'}, inplace=True)

# total late but highly important orders handled by each warehouse block
y = ecomm[(ecomm['Reached.on.Time_Y.N']==1) & (ecomm['Product_importance']=='high')].groupby('Warehouse_block').agg({'Reached.on.Time_Y.N':'count'})
y.rename(columns={'Reached.on.Time_Y.N':'Number of Highly Important Orders That Arrived Late'}, inplace=True)

# concatenate w,x & y
z = pd.concat([w,x,y], axis=1)
z['% of Late Orders (In Proportion Of Total Highly Important Orders Handled)'] = round(z['Number of Highly Important Orders That Arrived Late'] / z['Number of Highly Important Orders Handled'] *100, 2)
z['% of Late Orders (In Proportion Of Total Orders Handled)'] = round(z['Number of Highly Important Orders That Arrived Late'] / z['Total Number of Orders Handled'] *100, 2)
z.sort_values('% of Late Orders (In Proportion Of Total Orders Handled)', ascending=False)

#### From the charts and table above, we can infer that:
1. Warehouse block F resulted the highest number of late orders. It is understandable as it handled the highest number of orders, and this might due to the lack of workforce in warehouse block F inefficient workflow or lack of facilities to ease the work of the staff. Anyhow, more investigation and visit to warehouse block F needed to determine real issue of the high number of late orders.
2. If we only look at the data from the warehouse block A to D (warehouse E does not exist), we can clearly see that warehouse block A resulted the highest number of 'low importance' late orders, while warehouse block B resulted the highest number of 'medium importance' late orders and warehouse block A resulted the highest number of 'high importance' late orders.
3. Warehouse block A resulted a relatively high number of highly important but late orders (111), which is 18.02% [111/ 616*100] out of the total late but highly important orders (616). 
4. The table above also showed that warehouse block A  resulted the highest percentage of late but highly important order (6.06% @ 165/1833*100) in proportion of the total orders that the warehouse block A handled. Besides, warehouse block A handled 165 highly important orders but 111 of them were late for arrival, which is 67.27%. Hence, it is suggested to check whether warehouse block A is taking the make an evaluation on the workflow in warehouse block A or check the distance between warehouse block A to destination to determine whether warehouse block A is always to far locations. 

### Q4. What is the range of discount given for the products of different importance?

In [15]:
sns.boxplot(x=ecomm['Product_importance'],
            y=ecomm['Discount_offered'])
# sns.set(rc={'figure.figsize':(12,20)})
# plt.figure(figsize=(8,10));

#### From the boxplot above, we can infer that:
1. The minimum and the median of discount offered for all product importance are the same.
2. The range of discount for low and medium product importance are the same, which is 19-1=18. Meanwhile, the range of discount for high product importance is even larger, which is 39-1=38.There are many outliers for these categories. It is suggested that there need to have a maximum discount given for each of the categories. It is not advisable to give high discount to retain the customers as it might affect the company's revenue. It is recommended to improve the customer service and delivery service to retain the customers.

# 📌 Classification ML Algorithms - To Predict Shipment

In [16]:
# ecomm['Gender'] = ecomm.Gender.map({'F':0, 'M':1})
dummy1 = pd.DataFrame(pd.get_dummies(ecomm[['Warehouse_block', 'Mode_of_Shipment','Product_importance','Gender']]))
dummy2 = pd.DataFrame(pd.get_dummies(ecomm[['Customer_care_calls','Customer_rating','Prior_purchases']].astype(str)))
ecomm1 = pd.DataFrame(scale(ecomm[['Cost_of_the_Product','Discount_offered', 'Weight_in_gms']]),
                      columns=['Cost_of_the_Product','Discount_offered', 'Weight_in_gms'])
ecomm2 = pd.concat([ecomm1,dummy1,dummy2,ecomm[['Reached.on.Time_Y.N']]],axis=1)

In [17]:
ecomm2.columns

In [18]:
ecomm2.shape

In [19]:
with pd.option_context('display.max_columns',None):
    display(HTML(ecomm2.sample(5).to_html()))

In [20]:
# split data into output and input
X = ecomm2.iloc[:,:-1] # inputs
Y = ecomm2['Reached.on.Time_Y.N'] # outputs

# split data into train data and test data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25,shuffle=True) # test data is of 25% of the total records

In [21]:
with pd.option_context('display.max_columns',None):
    display(HTML(X_train.sample(5).to_html()))

In [22]:
# append different classification models into classifiers array 
classifiers=[]
LR_model = LogisticRegression()
classifiers.append(LR_model)
KNN_model = KNeighborsClassifier(n_neighbors=11, metric='euclidean')
classifiers.append(KNN_model)
DT_model = DT(criterion = 'entropy',max_depth=4) # Decision Tree
classifiers.append(DT_model)
RF_model = RT(n_jobs=2, n_estimators=99, criterion="entropy")
classifiers.append(RF_model)
GNB_model = GB() 
classifiers.append(GNB_model)
SVC_model = SVC(kernel = "linear")
classifiers.append(SVC_model)
XGB_model = xgboost.XGBClassifier()
classifiers.append(XGB_model)

In [23]:
classifiers

In [24]:
accuracy_train = []
accuracy_test = []
for clf in classifiers:
    clf.fit(X_train, Y_train)
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    acc_train = accuracy_score(Y_train, pred_train)
    acc_test = accuracy_score(Y_test, pred_test)
    accuracy_train.append(acc_train)
    accuracy_test.append(acc_test)
    #print("Accuracy of %s is %s"%(clf, acc))

In [25]:
accuracy_result = pd.DataFrame(data={'Model':['LR','KNN','Decision Tree','Random Forest','Gaussian Bayes','SVM','XGB'],
                                     'Training Accuracy':accuracy_train,
                                     'Testing Accuracy':accuracy_test})
accuracy_result.sort_values('Testing Accuracy',ascending=False)

In [26]:
#ROC Curve
from sklearn.metrics import roc_curve
y_pred_prob1 = classifiers[0].predict_proba(X_test)[:,1]
fpr1 , tpr1, thresholds1 = roc_curve(Y_test, y_pred_prob1)

y_pred_prob2 = classifiers[1].predict_proba(X_test)[:,1]
fpr2 , tpr2, thresholds2 = roc_curve(Y_test, y_pred_prob2)

y_pred_prob3 = classifiers[2].predict_proba(X_test)[:,1]
fpr3 , tpr3, thresholds3 = roc_curve(Y_test, y_pred_prob3)

y_pred_prob4 = classifiers[3].predict_proba(X_test)[:,1]
fpr4 , tpr4, thresholds4 = roc_curve(Y_test, y_pred_prob4)

y_pred_prob5 = classifiers[4].predict_proba(X_test)[:,1]
fpr5 , tpr5, thresholds5 = roc_curve(Y_test, y_pred_prob5)

y_pred_prob6 = classifiers[6].predict_proba(X_test)[:,1]
fpr6 , tpr6, thresholds6 = roc_curve(Y_test, y_pred_prob6)

plt.figure(figsize = (20,20))
plt.plot([0,1],[0,1], 'k--')
plt.plot(fpr1, tpr1, label= "LR")
plt.plot(fpr2, tpr2, label= "KNN")
plt.plot(fpr3, tpr3, label= "DT")
plt.plot(fpr4, tpr4, label= "RF")
plt.plot(fpr5, tpr5, label= "NB")
plt.plot(fpr6, tpr6, label= "XGB")
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title('Receiver Operating Characteristic')
plt.show()

## 📌 Selected Model : Decision Tree

In [27]:
DT_model = DT(criterion = 'entropy',max_depth=4) 
DT_model.fit(X_train,Y_train)

In [28]:
fig, ax = plt.subplots(figsize=(20, 20), facecolor='w')
tree.plot_tree(DT_model,
               feature_names=np.array(X.columns),
               class_names=['0','1'],
               filled=True,
               ax=ax);

In [29]:
pred_train_DT = DT_model.predict(X_train)
pred_test_DT = DT_model.predict(X_test)

In [30]:
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
train_cnf_matrix = confusion_matrix(Y_train,pred_train_DT)
train_counts = ["{0:0.0f}".format(value) for value in train_cnf_matrix.flatten()]
train_percentage = ["{0:.2%}".format(value) for value in train_cnf_matrix .flatten()/np.sum(train_cnf_matrix)]
train_labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,train_counts,train_percentage)]
train_labels = np.asarray(train_labels).reshape(2,2)
plt.figure(figsize = (16,5))
sns.heatmap(train_cnf_matrix, annot=train_labels, fmt='', cmap='coolwarm')
plt.title('Confusion matrix for train data (DT model)',fontsize=16);

In [31]:
group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
test_cnf_matrix = confusion_matrix(Y_test,pred_test_DT)
test_counts = ["{0:0.0f}".format(value) for value in test_cnf_matrix.flatten()]
test_percentage = ["{0:.2%}".format(value) for value in test_cnf_matrix .flatten()/np.sum(test_cnf_matrix)]
test_labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,test_counts,test_percentage)]
test_labels = np.asarray(test_labels).reshape(2,2)
plt.figure(figsize = (16,5))
sns.heatmap(test_cnf_matrix, annot=test_labels, fmt='', cmap='Blues')
plt.title('Confusion matrix for test data (DT model)',fontsize=16);

In [32]:
Y_test.value_counts()

In [33]:
len(pred_test_DT[pred_test_DT==1])

In [34]:
len(pred_test_DT[pred_test_DT==0])

In [35]:
print(f"{classification_report(Y_train,pred_train_DT)}")

In [36]:
print(f"{classification_report(Y_test,pred_test_DT)}")